## Pokemon types, aka "super effective"

In Pokemon, `attacks` have 1 of 18 [types](https://pokemondb.net/type),
like `fire` or `water`. The `monsters` themselves have single or 
[dual type](https://pokemondb.net/type/dual), eg `fire` or `fire/bug`.
Note that `fire/bug` and `bug/fire` mean the same thing.

Some types are stronger or weaker against others. 
For example, `water` attacks deal 2x damage on `fire` monsters 
and 1/2 against `water`.
For dual-type monsters, attacks apply to each type then multiply.
So `water` deals 4x damage to `fire/ground`
and 1x (regular) damage to `fire/water`.

**Goal**: programatically find the strongest and weakest of the 
18 + 18*17/2 = 171 single or dual types.

Ideas
1. Replicate the [cumulative scores](https://pokemondb.net/type/dual).
2. type coverage: enumerate all 4-tuples of types that 2x all types
3. input pokemon name, find its single/dual type, recommend moveset maximizing coverage, eg ice monster should carry ground attack to counter rock and steel STABers


In [1]:
data = {
#     'normal': {
#         'rock': 1/2,
#         'ghost': 0,
#         'steel': 1/2
#     },
    'fire': {
        'fire':1/2,
        'water':1/2,
        'grass':2,        
    },
    'water': {
        'fire':2,
        'water':1/2,
        'grass':1/2,
    },
    'electric': {
        'water':2,
        'electric':1/2,
        'grass':1/2,
    },
    'grass': {
        'fire':1/2,
        'water':2,
        'grass':1/2,
    }   
}


In [2]:
from collections import defaultdict


def test_build_effectiveness(f):
    d_in = {
        'fire':{'water':1/2}, 
        'water':{'fire':2},
    }
    d_out = {
        'fire':{'fire':1,'water':2},
        'water':{'fire':1/2,'water':1},
        ('fire','water'):{'fire':1/2,'water':2}
    }
    assert dict(f(d_in)) == dict(d_out)
    print('test_build_effectiveness passed')
    
    
def build_effectiveness(data):
    """ data = map of atk element -> def element -> effectiveness
    """
    # map each type or dual type to a dict of type effectiveness
    resist = defaultdict(dict) # map single/dual type -> atk type -> effectiveness
    all_types = data.keys()
    
    for atk_t in all_types:
        for t1 in all_types:
            # single type
            eff = data[atk_t][t1] if t1 in data[atk_t] else 1        
            resist[t1][atk_t] = eff
            # dual types
            for t2 in all_types:
                if t1 >= t2: # t1,t2 is the same as t2,t1. Keep the first alphabetically.
                    continue
                eff = (
                    (data[atk_t][t1] if t1 in data[atk_t] else 1)
                    * (data[atk_t][t2] if t2 in data[atk_t] else 1)
                )
                resist[(t1,t2)][atk_t] = eff
    return resist

test_build_effectiveness(build_effectiveness)

effectives = build_effectiveness(data)


test_build_effectiveness passed


In [3]:
effectives

defaultdict(dict,
            {'fire': {'fire': 0.5, 'water': 2, 'electric': 1, 'grass': 0.5},
             ('fire', 'water'): {'fire': 0.25,
              'water': 1.0,
              'electric': 2,
              'grass': 1.0},
             ('fire', 'grass'): {'fire': 1.0,
              'water': 1.0,
              'electric': 0.5,
              'grass': 0.25},
             'water': {'fire': 0.5, 'water': 0.5, 'electric': 2, 'grass': 2},
             'electric': {'fire': 1, 'water': 1, 'electric': 0.5, 'grass': 1},
             ('electric', 'fire'): {'fire': 0.5,
              'water': 2,
              'electric': 0.5,
              'grass': 0.5},
             ('electric', 'water'): {'fire': 0.5,
              'water': 0.5,
              'electric': 1.0,
              'grass': 2},
             ('electric', 'grass'): {'fire': 2,
              'water': 0.5,
              'electric': 0.25,
              'grass': 0.5},
             'grass': {'fire': 2, 'water': 0.5, 'electric': 0.5, 'grass'